
Starting with 1 and spiralling anticlockwise in the following way, a square spiral with side length 7 is formed.


$$
|37| 36| 35| 34| 33| 32| 31| \\ 
|38| 17| 16| 15| 14| 13| 30|  \\
|39| 18|  5|  4|  3| 12| 29|  \\
|40| 19|  6|  1|  2| 11| 28|  \\
|41| 20|  7|  8|  9| 10| 27|  \\ 
|42| 21| 22| 23| 24| 25| 26| \\
|43| 44| 45| 46| 47| 48| 49|
$$


It is interesting to note that the odd squares lie along the bottom right diagonal, but what is more interesting is that 8 out of the 13 numbers lying along both diagonals are prime; that is, a ratio of 8/13 ≈ 62%.

If one complete new layer is wrapped around the spiral above, a square spiral with side length 9 will be formed. If this process is continued, what is the side length of the square spiral for which the ratio of primes along both diagonals first falls below 10%?

#Approach:
####1. Create matrix
####2. Fill matrix
####3. Check if diagonals are prime
####4. Calculate %

In [1]:
import numpy as np
import eulertools

In [48]:
def create_matrix(n):
    return np.zeros((n,n))
def move(mat,position,direction):
    maxval = len(mat)
    minval = 0

    if direction == 'N':
        next_move = (-1,0)
    elif direction == 'E':
        next_move = (0,1)
    elif direction == 'S':
        next_move = (1,0)
    elif direction == 'W':
        next_move = (0,-1)
    else:
        raise ValueError('Only NWSE available as direction')
    new_position = (position[0]+next_move[0],position[1]+next_move[1])
    if minval <= new_position[0] < maxval:
        if minval <= new_position[1] <maxval:
            return new_position
    raise ValueError('Movement out of bounds')
        
    
def increase_size(mat):
    return np.lib.pad(mat,1,'constant')

In [49]:
def fill_matrix2(matr):
    '''receives a filled matrix of sides n. Increments the side to n+2, transfers the matrix to the center
    of the new matrix, and fills the reamining squares'''
    mat = increase_size(matr)
    n2 = len(mat)
    position = (n2-2, n2-2)
    moves = list('E'  + (n2-2)*'N' + (n2-1)*'W' + (n2-1)*'S' + (n2-1)*'E')
    vals = [x for x in range(2,len(mat)**2 +1) if x > mat.max()]
    vals = vals[::-1]
    for direction in moves:
        #print(position, direction, '\n',mat,'\n', n2, 'before updating')
        position = move(mat,position,direction)
        mat[position[0]][position[1]] = vals.pop()
        #print(position, direction,'\n', mat,'\n', n2, 'after updating')

    return mat

In [50]:
def fill_matrix(mat):
    center = 1
    
    mat[np.floor(len(mat)/2)][np.floor(len(mat)/2)] = center
    position = (np.floor(len(mat)/2),np.floor(len(mat)/2))
    vals = [x for x in range(2,len(mat)**2 +1)]
    vals = vals[::-1]
    for i in range(1,len(mat)+1,2):
        if i != 1:
        
            moves = 'E'  + (i-2)*'N' + (i-1)*'W' + (i-1)*'S' + (i-1)*'E'
            moves = list(moves)
            for direction in moves:
                #print(position, direction, '\n',mat,'\n', i, 'before updating')
                position = move(mat,position,direction)
                mat[position[0]][position[1]] = vals.pop()
                #print(position, direction,'\n', mat,'\n', i, 'after updating')
            
    return  mat
        

In [ ]:
dict_mats = {}


In [4]:
a = create_matrix(3)

In [7]:
a=fill_matrix(a)
a

array([[ 5.,  4.,  3.],
       [ 6.,  1.,  2.],
       [ 7.,  8.,  9.]])

TypeError: unhashable type: 'numpy.ndarray'

In [8]:
def check_diagonals(mat, primes):
    if mat.max() < primes.max():
        primes = primes
    else:
        primes = eulertools.primesfrom2to(len(10*mat)*len(10*mat)))
    ct = 0
    isprime = []
    for n in range(0,len(mat)):
        if int(mat[n][n]) in primes:
            isprime.append(mat[n][n])
        ct+=1
    m = len(mat)-1
    for n in range(0,len(mat)):
        if int(mat[n][m]) in primes:
            isprime.append(mat[n][n])
        ct+=1
        m -=1
    #print(ct-1)
    return len(isprime), ct-1, primes

In [63]:
b,c = check_diagonals(a)
b/c

0.6153846153846154

In [18]:
#intialize
mat = create_matrix(3)
mat = fill_matrix(mat)
primes = eulertools.primesfrom2to(1000000)

ratio = 1

while ratio >= 0.1:
    
    mat = fill_matrix2(mat)
    pr, diag, primes = check_diagonals(mat,primes)
    ratio = pr/diag

    
    

KeyboardInterrupt: 

In [2]:
primes = eulertools.primesfrom2to(1000000)

In [23]:
import cProfile as cp

In [12]:
def alternate(primes, val):
    '''Deduce the way the numbers increase as a function of the increased side. Apply that knowdledge to 
    infer the turning point'''
    #intialize
    diagonals = [1,3,5,7,9]
    dist = len(diagonals)
    pmax = primes.max()
    maxd = max(diagonals)
    ratio = 1
    pr_list = [1 if x in primes else 0 for x in diagonals]
    n=5
    
    #loop
    while ratio > val:
    
        if maxd < pmax:
            primes = primes
        else:
            primes = eulertools.primesfrom2to(100*pmax)
            pmax = primes.max()
        ext_list = [maxd+(n-1), maxd+2*(n-1), maxd + 3*(n-1), maxd+4*(n-1)]
        ext_pr = [1  if x in primes else 0 for x in ext_list]
        
        diagonals.extend(ext_list)
        pr_list.extend(ext_pr)
        
        maxd = maxd + 4*(n-1)
        dist +=4
        ratio = sum(pr_list)/dist
        
        n +=2
        
    return n, ratio, maxd, diagonals, pr_list
    
    
    
   # pr_list = [1 for x in diagonals if x in primes else 0]
    

In [61]:
cp.run('alternate(primes,0.2)')

         774 function calls in 1.148 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.011    0.011    1.147    1.147 <ipython-input-12-769ac68c0a27>:1(alternate)
      153    1.124    0.007    1.124    0.007 <ipython-input-12-769ac68c0a27>:21(<listcomp>)
        1    0.007    0.007    0.007    0.007 <ipython-input-12-769ac68c0a27>:9(<listcomp>)
        1    0.000    0.000    1.147    1.147 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 _methods.py:25(_amax)
        1    0.000    0.000    1.148    1.148 {built-in method exec}
      153    0.000    0.000    0.000    0.000 {built-in method len}
        1    0.000    0.000    0.000    0.000 {built-in method max}
      153    0.004    0.000    0.004    0.000 {built-in method sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
      306    0.001    0.000    0.001    0.000 {method 'extend' of 'list' 

In [65]:
cp.run('alternate2(primes,0.1)')

         66 function calls in 565.538 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  561.673  561.673  565.537  565.537 <ipython-input-63-a9ca4eb49247>:1(alternate2)
        1    0.007    0.007    0.007    0.007 <ipython-input-63-a9ca4eb49247>:10(<listcomp>)
        1    0.000    0.000  565.537  565.537 <string>:1(<module>)
        2    0.000    0.000    0.030    0.015 _methods.py:25(_amax)
        1    3.425    3.425    3.828    3.828 eulertools.py:2(primesfrom2to)
        1    0.000    0.000    0.271    0.271 fromnumeric.py:1484(nonzero)
        1    0.000    0.000    0.000    0.000 index_tricks.py:231(_retval)
        1    0.001    0.001    0.060    0.060 index_tricks.py:251(__getitem__)
        1    0.000    0.000    0.071    0.071 numeric.py:141(ones)
        1    0.000    0.000    0.000    0.000 numerictypes.py:1015(<listcomp>)
        1    0.000    0.000    0.000    0.000 numerictypes.py:1016(<listcomp>)


KeyboardInterrupt: 

In [63]:
def alternate2(primes, val):
    '''Deduce the way the numbers increase as a function of the increased side. Apply that knowdledge to 
    infer the turning point'''
    #intialize
    diagonals = [1,3,5,7,9]
    dist = len(diagonals)
    pmax = primes.max()
    maxd = max(diagonals)
    ratio = 1
    pr_list = [1 if x in primes else 0 for x in diagonals]
    
    prs = sum(pr_list)
    n=3
    
    #loop
    while ratio > val:
        n+=2
    
        if maxd < pmax:
            primes = primes
        else:
            primes = eulertools.primesfrom2to(100*pmax)
            pmax = primes.max()
        
        for i in range(1,5):
            value = maxd + i *(n-1)
            #diagonals.append(value)
            if value in primes:
                prs +=1
        #ext_list = [maxd+(n-1), maxd+2*(n-1), maxd + 3*(n-1), maxd+4*(n-1)]
        #ext_pr = [1  if x in primes else 0 for x in ext_list]
        
        #diagonals.extend(ext_list)
        #pr_list.extend(ext_pr)
        
        maxd = maxd + 4*(n-1)
        dist +=4
        ratio = prs/dist
    
        
    
        
    return n, ratio, prs, maxd, dist#, diagonals
    
    
   # pr_list = [1 for x in diagonals if x in primes else 0]
    

In [26]:
from numba import jit


In [58]:
t[1]

0.09090909090909091

In [ ]:
alternate2(primes, 0.1)

In [51]:
a = create_matrix(17)

In [53]:
a = fill_matrix(a)
a

array([[ 257.,  256.,  255.,  254.,  253.,  252.,  251.,  250.,  249.,
         248.,  247.,  246.,  245.,  244.,  243.,  242.,  241.],
       [ 258.,  197.,  196.,  195.,  194.,  193.,  192.,  191.,  190.,
         189.,  188.,  187.,  186.,  185.,  184.,  183.,  240.],
       [ 259.,  198.,  145.,  144.,  143.,  142.,  141.,  140.,  139.,
         138.,  137.,  136.,  135.,  134.,  133.,  182.,  239.],
       [ 260.,  199.,  146.,  101.,  100.,   99.,   98.,   97.,   96.,
          95.,   94.,   93.,   92.,   91.,  132.,  181.,  238.],
       [ 261.,  200.,  147.,  102.,   65.,   64.,   63.,   62.,   61.,
          60.,   59.,   58.,   57.,   90.,  131.,  180.,  237.],
       [ 262.,  201.,  148.,  103.,   66.,   37.,   36.,   35.,   34.,
          33.,   32.,   31.,   56.,   89.,  130.,  179.,  236.],
       [ 263.,  202.,  149.,  104.,   67.,   38.,   17.,   16.,   15.,
          14.,   13.,   30.,   55.,   88.,  129.,  178.,  235.],
       [ 264.,  203.,  150.,  105.,   68.,   39.

In [98]:
a

array([[ 37.,  36.,  35.,  34.,  33.,  32.,  31.],
       [ 38.,  17.,  16.,  15.,  14.,  13.,  30.],
       [ 39.,  18.,   5.,   4.,   3.,  12.,  29.],
       [ 40.,  19.,   6.,   1.,   2.,  11.,  28.],
       [ 41.,  20.,   7.,   8.,   9.,  10.,  27.],
       [ 42.,  21.,  22.,  23.,  24.,  25.,  26.],
       [ 43.,  44.,  45.,  46.,  47.,  48.,  49.]])